# Calculadora de k-meros más abundantes en secuencia .fasta

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import os

# Step 1: Read sequence from FASTA file
def read_fasta(filepath):
    """
    Reads a sequence from a FASTA file.
    Skips lines starting with '>'.
    """
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"The file {filepath} does not exist.")
    
    sequence = []
    with open(filepath, 'r') as file:
        for line in file:
            if line.startswith(">"):
                continue  # Skip header lines
            sequence.append(line.strip())
    return ''.join(sequence)

# Step 2: Extract k-mers
def extract_kmers(sequence, k):
    """
    Extracts all k-mers from a given sequence.
    """
    return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]

# Step 3: Count k-mer occurrences
def count_kmers(kmers):
    """
    Counts the occurrences of each k-mer in the list.
    """
    return Counter(kmers)

# Step 4: Find the top N frequent k-mers
def find_top_kmers(kmer_counts, n=7):
    """
    Finds the top N most frequent k-mers.
    Returns a list of tuples: [(k-mer, count), ...].
    """
    top_kmers = kmer_counts.most_common(n)
    
    # Debugging step
    if not all(isinstance(item, tuple) and len(item) == 2 for item in top_kmers):
        print("Unexpected structure in top_kmers:", top_kmers)
        raise ValueError("top_kmers must be a list of (k-mer, count) tuples")
    
    return top_kmers

# Step 5: Store results in a DataFrame
def kmer_results_to_dataframe(filepath, k_values, n=7):
    """
    Computes top N frequent k-mers for multiple k-values and stores results in a DataFrame.
    Includes normalized weight (relative frequency) for each k-mer.
    """
    results = []
    sequence = read_fasta(filepath)  # Read the sequence once

    for k in k_values:
        kmers = extract_kmers(sequence, k)  # Extract k-mers
        kmer_counts = count_kmers(kmers)  # Count occurrences
        total_kmers = sum(kmer_counts.values())  # Total k-mers for normalization

        top_kmers = find_top_kmers(kmer_counts, n)  # Get top N k-mers

        for rank, (kmer, count) in enumerate(top_kmers, start=1):
            normalized_weight = count / total_kmers  # Compute normalized weight

            results.append({
                "k-mer": kmer,
                "Count": count,
                "Normalized Weight": normalized_weight
            })

    return pd.DataFrame(results)


In [2]:
%%time
if __name__ == "__main__":
    import time
    # Ejemplo
    # Path to your FASTA file
    fasta_file = "/files/Mariel/Tesis_Mariel/data/Examples/Neisseria_gonorrhoeae_SRR1661157.contigs.fasta"
    
    k_values = np.arange(1, 4)  # Range of k-mer lengths
    n = 4  # Number of most frequent k-mers to find

    start_time = time.time()
    try:
        df = kmer_results_to_dataframe(fasta_file, k_values, n)
        print("Top k-mers DataFrame:")
        print(df)
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        print(f"Elapsed time: {time.time() - start_time:.2f} seconds")


Top k-mers DataFrame:
   k-mer   Count  Normalized Weight
0      C  565136           0.263903
1      G  559775           0.261399
2      T  510661           0.238464
3      A  505885           0.236234
4     CG  196108           0.091577
5     GC  188700           0.088118
6     TT  180212           0.084154
7     AA  178621           0.083411
8    TTT   68507           0.031991
9    AAA   68494           0.031985
10   CCG   60738           0.028363
11   CGG   59844           0.027945
Elapsed time: 1.13 seconds
CPU times: user 1.04 s, sys: 97.7 ms, total: 1.13 s
Wall time: 1.13 s
